<a href="https://colab.research.google.com/github/yongminkim0501/DeepLearning/blob/main/VoiceRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 기본 데이터 분석 및 과학 계산 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import os
# 음성 처리 관련 라이브러리
import librosa
import librosa.display
import soundfile as sf

# 딥러닝 라이브러리
import tensorflow as tf
from tensorflow import keras
from transformers import pipeline, AutoModelForAudioClassification, AutoFeatureExtractor

음성인식 라이브러리

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


구글드라이브 연결

In [4]:
cd /content/drive/MyDrive/VoiceRecognition

/content/drive/MyDrive/VoiceRecognition


In [9]:
tar_file = '/content/drive/MyDrive/VoiceRecognition/download.tar'
extract_dir = '/content/drive/MyDrive/VoiceRecognition'

!mkdir -p {extract_dir}
!tar -xf {tar_file} -C {extract_dir}

In [10]:
cd /content/drive/MyDrive/VoiceRecognition/KoreanVoice

/content/drive/MyDrive/VoiceRecognition/KoreanVoice


In [11]:
# 파일 조각들을 하나로 합치기
!cat KsponSpeech_01.zip.part* > KsponSpeech_01.zip

# zip 파일 압축 해제
!unzip KsponSpeech_01.zip -d /content/drive/MyDrive/VoiceRecognition/UnpackedKoreanVoice

Archive:  KsponSpeech_01.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of KsponSpeech_01.zip or
        KsponSpeech_01.zip.zip, and cannot find KsponSpeech_01.zip.ZIP, period.


In [13]:
!ls -la /content/drive/MyDrive/VoiceRecognition/KoreanVoice/KsponSpeech_01.zip*

-rw------- 1 root root 15301122689 Mar 25 04:29 /content/drive/MyDrive/VoiceRecognition/KoreanVoice/KsponSpeech_01.zip
-rw------- 1 root root  1073741824 Mar 24 09:12 /content/drive/MyDrive/VoiceRecognition/KoreanVoice/KsponSpeech_01.zip.part0
-rw------- 1 root root  1073741824 Mar 24 09:14 /content/drive/MyDrive/VoiceRecognition/KoreanVoice/KsponSpeech_01.zip.part1073741824
-rw------- 1 root root  1073741824 Mar 24 09:30 /content/drive/MyDrive/VoiceRecognition/KoreanVoice/KsponSpeech_01.zip.part10737418240
-rw------- 1 root root  1073741824 Mar 24 09:31 /content/drive/MyDrive/VoiceRecognition/KoreanVoice/KsponSpeech_01.zip.part11811160064
-rw------- 1 root root  1073741824 Mar 24 09:32 /content/drive/MyDrive/VoiceRecognition/KoreanVoice/KsponSpeech_01.zip.part12884901888
-rw------- 1 root root  1073741824 Mar 24 09:33 /content/drive/MyDrive/VoiceRecognition/KoreanVoice/KsponSpeech_01.zip.part13958643712
-rw------- 1 root root   268737153 Mar 24 09:35 /content/drive/MyDrive/VoiceRecogn

In [14]:
!cat KsponSpeech_01.zip.part* > KsponSpeech_complete.zip

In [15]:
!ls -la KsponSpeech_complete.zip

-rw------- 1 root root 15301122689 Mar 25 04:37 KsponSpeech_complete.zip


In [16]:
!mkdir -p KsponSpeech_extracted

In [17]:
!ls -la KsponSpeech_extracted

total 0


In [ ]:
!unzip KsponSpeech_complete.zip -d KsponSpeech_extracted

In [19]:
!file KsponSpeech_01.zip.part0

KsponSpeech_01.zip.part0: Zip archive data, at least v2.0 to extract, compression method=store


In [ ]:
!apt-get install p7zip-full -y && 7z x KsponSpeech_01.zip.part0 -oKsponSpeech_extracted && ls -la KsponSpeech_extracted

------------------------------ 데이터 처리를 위한 기본 과정 (실패) 다시 해야함
------------------

In [2]:
# 샘플링 레이트 설정
fs = 22050  # 일반적인 오디오 샘플링 레이트

# 임시 음성 데이터 생성 (약 1초 길이의 신호)
duration = 1.0  # 1초
t = np.linspace(0, duration, int(fs * duration), endpoint=False)

# 여러 주파수 성분을 가진 신호 생성
# 기본 주파수 (예: 말소리의 기본 주파수 범위에 해당)
f0 = 120  # Hz (남성 목소리의 기본 주파수에 가까움)
# 하모닉스 추가
data = 0.5 * np.sin(2 * np.pi * f0 * t)  # 기본 주파수
data += 0.3 * np.sin(2 * np.pi * 2 * f0 * t)  # 2배 하모닉
data += 0.15 * np.sin(2 * np.pi * 3 * f0 * t)  # 3배 하모닉
data += 0.05 * np.sin(2 * np.pi * 4 * f0 * t)  # 4배 하모닉

# 약간의 노이즈 추가
noise = np.random.normal(0, 0.05, len(data))
data += noise

In [3]:
#data = 0                 # 원래는 특정 데이터가 들어가야함
n_fft = 2048
hop = n_fft // 4
stft = librosa.stft(data, n_fft=n_fft, window='hann', hop_length=hop, center=True)
print('stft.shape', stft.shape)
# STFT 계산(출력차원: (n_fft//2) + 1 , T ) =(1025,13)
# T = (L - n_fft) // hop_length + 1(=구간개수=13)

# Mel Filter Bank 계수 계산
mel_f = librosa.filters.mel(sr=fs, n_fft=n_fft, n_mels=128)
print('mel_f.shape',mel_f.shape)

stft.shape (1025, 44)
mel_f.shape (128, 1025)


MFCC (Mel Frequency Cepstral Coefficients)의 약자
음성 신호의 특징을 추출하는 기술
1. STFT(Short Time Fourier Transform)에 의해 주어진 음성 신호를 작은 프레임 단위로 나누어서 주파수 영역의 데이터로 변환
2. Mel Filter Bank로 멜 스펙트럼을 계산
3. 로그 스케일링하고 DCT(Discrete Cosine Transform)을 수행
4. 이를 이용하여 해당 프레임의 특징을 추출
-> 이렇게 추출된 특징은 일반적으로 MFCC 계수라고 부름

출처 : https://wikidocs.net/193361

In [3]:
os.chdir("/content/drive/MyDrive/VoiceRecognition")
file_path = "/content/drive/MyDrive/VoiceRecognition"
train_path = file_path + "/train"
test_path = file_path + "/test"

기본적인 데이터를 불러오기 위한 주소 정리

In [ ]:
y,sr = librosa.load(file_path, sr=None) #sr=None은 원본 샘플링 레이트 유지

voice_normalized = librosa.util.normalize(y)
target_sr = 0                           # 리샘플링 관련 샘플링 관련하여 필요 시, 데이터 전처리하며 분석하여 진행
y_resampled = librosa.resample(y, orig_sr = sr, target_sr = target_sr)

리샘플링 관련 코드

In [ ]:
import scipy.signal as signal
b, a = signal.butter(5, 100/(sr/2), 'highpass')
y_filtered = signal.filtfilt(b, a, y)   # highpass 필터를 이용한 간단한 노이즈 필터링
                                        # 또는 스펙트럴 서브 트랙션 등 더 복잡한 노이즈 제거 방법 사용

노이즈 필터링 관련된 부분

In [7]:
import tensorflow as tf
from transformers import TFBertForSequenceClassification, BertTokenizer

# 모델 및 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained("snunlp/KR-BERT-char16424")
model = TFBertForSequenceClassification.from_pretrained(
    "snunlp/KR-BERT-char16424",
    num_labels=3,
    from_pt=True  # PyTorch 가중치를 TensorFlow로 변환
)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import librosa
import os
from transformers import WhisperProcessor, TFWhisperForConditionalGeneration
from transformers import WhisperTokenizer
from datasets import Dataset, Audio
import evaluate

class WhisperFineTuner:
    def __init__(self, model_name="openai/whisper-small", batch_size=16):
        self.model_name = model_name
        self.batch_size = batch_size

        # 모델 및 프로세서 초기화
        print(f"모델 로딩 중: {model_name}")
        self.processor = WhisperProcessor.from_pretrained(model_name)
        self.tokenizer = WhisperTokenizer.from_pretrained(model_name, language="ko", task="transcribe")

        # 모델 로드 (from_pt=True로 PyTorch 가중치 변환)
        self.model = TFWhisperForConditionalGeneration.from_pretrained(
            model_name, from_pt=True
        )

        # WER(Word Error Rate) 메트릭 초기화
        self.metric = evaluate.load("wer")

    def prepare_dataset(self, data_path, csv_file=None):
        """
        데이터셋 준비
        - data_path: 오디오 파일이 있는 디렉토리 경로
        - csv_file: 오디오 파일 경로와 텍스트 전사 내용이 있는 CSV 파일
                   (columns: ['file_path', 'transcription'])
        """
        if csv_file:
            # CSV에서 데이터 로드
            df = pd.read_csv(csv_file)
            data_dict = {
                "file_path": df["file_path"].tolist(),
                "transcription": df["transcription"].tolist()
            }
        else:
            # CSV가 없을 경우 디렉토리에서 오디오 파일만 수집
            file_paths = [os.path.join(data_path, f) for f in os.listdir(data_path)
                         if f.endswith(('.wav', '.mp3', '.flac'))]

            # 이 경우 전사 데이터가 없으므로 빈 문자열로 초기화 (준지도 학습에 활용 가능)
            data_dict = {
                "file_path": file_paths,
                "transcription": [""] * len(file_paths)
            }

        # Hugging Face 데이터셋 생성
        dataset = Dataset.from_dict(data_dict)

        # 오디오 로드 설정
        dataset = dataset.cast_column("file_path", Audio(sampling_rate=16000))

        # 전처리 함수 정의
        def prepare_dataset(examples):
            # 오디오 파일 로드
            audio = examples["file_path"]

            # 샘플링 레이트 확인 및 변환
            if isinstance(audio, dict) and "array" in audio:
                speech = audio["array"]
                sampling_rate = audio["sampling_rate"]
            else:
                # librosa로 직접 로드
                speech, sampling_rate = librosa.load(examples["file_path"], sr=16000)

            # Whisper 모델용 입력 준비
            input_features = self.processor.feature_extractor(
                speech, sampling_rate=sampling_rate, return_tensors="tf"
            ).input_features[0]

            # 레이블(전사 텍스트) 토큰화
            label = examples["transcription"]
            labels = self.tokenizer(label, return_tensors="tf").input_ids[0]

            return {
                "input_features": input_features,
                "labels": labels
            }

        # 데이터셋에 전처리 함수 적용
        processed_dataset = dataset.map(prepare_dataset)

        # 학습 및 검증 데이터 분할
        train_test_split = processed_dataset.train_test_split(test_size=0.1)

        return train_test_split

    def compute_metrics(self, pred):
        """평가 메트릭 계산"""
        pred_ids = np.argmax(pred.predictions, axis=-1)

        # 레이블과 예측을 디코딩
        pred_str = self.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
        label_ids = np.where(pred.label_ids != -100, pred.label_ids, self.tokenizer.pad_token_id)
        label_str = self.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

        # WER 계산
        wer = self.metric.compute(predictions=pred_str, references=label_str)

        return {"wer": wer}

    def data_collator(self, features):
        """데이터 배치 처리"""
        input_features = [feature["input_features"] for feature in features]
        labels = [feature["labels"] for feature in features]

        # 패딩 처리
        batch = {
            "input_features": tf.stack(input_features),
            "labels": tf.keras.preprocessing.sequence.pad_sequences(
                labels, padding="post", value=self.tokenizer.pad_token_id
            )
        }

        return batch

    def finetune(self, dataset_dict, output_dir="./whisper-finetuned-korean", epochs=3):
        """모델 파인튜닝"""
        # 옵티마이저 및 학습 설정
        optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)

        # 모델 컴파일
        self.model.compile(optimizer=optimizer)

        # 데이터셋을 tf.data.Dataset으로 변환
        def to_tf_dataset(dataset):
            def gen():
                for item in dataset:
                    yield {
                        "input_features": item["input_features"],
                        "labels": item["labels"]
                    }

            return tf.data.Dataset.from_generator(
                gen,
                output_signature={
                    "input_features": tf.TensorSpec(shape=(80, None), dtype=tf.float32),
                    "labels": tf.TensorSpec(shape=(None,), dtype=tf.int32)
                }
            ).padded_batch(self.batch_size)

        train_dataset = to_tf_dataset(dataset_dict["train"])
        eval_dataset = to_tf_dataset(dataset_dict["test"])

        # 모델 학습
        callbacks = [
            tf.keras.callbacks.ModelCheckpoint(
                filepath=os.path.join(output_dir, "checkpoint-{epoch}"),
                save_weights_only=True
            ),
            tf.keras.callbacks.EarlyStopping(
                monitor="val_loss", patience=2, restore_best_weights=True
            )
        ]

        # 모델 학습
        history = self.model.fit(
            train_dataset,
            validation_data=eval_dataset,
            epochs=epochs,
            callbacks=callbacks
        )

        # 모델 저장
        self.model.save_pretrained(output_dir)
        self.processor.save_pretrained(output_dir)

        return history

    def evaluate(self, test_dataset):
        """모델 평가"""
        predictions = []
        references = []

        for batch in test_dataset:
            # 추론
            pred = self.model.generate(
                input_features=batch["input_features"],
                language="ko",
                task="transcribe"
            )

            # 디코딩
            decoded_preds = self.tokenizer.batch_decode(pred, skip_special_tokens=True)
            decoded_labels = self.tokenizer.batch_decode(
                batch["labels"],
                skip_special_tokens=True
            )

            predictions.extend(decoded_preds)
            references.extend(decoded_labels)

        # WER 계산
        wer = self.metric.compute(predictions=predictions, references=references)

        return {"wer": wer}


# 사용 예시
if __name__ == "__main__":
    # 파인튜너 초기화
    finetuner = WhisperFineTuner(model_name="openai/whisper-small")

    # 데이터셋 준비
    # 참고: 실제 파일 경로와 CSV 파일을 사용해야 함
    dataset = finetuner.prepare_dataset(
        data_path="./korean_audio_data",
        csv_file="./transcriptions.csv"
    )

    # 모델 파인튜닝
    history = finetuner.finetune(dataset, epochs=5)

    # 모델 평가
    metrics = finetuner.evaluate(dataset["test"])
    print(f"Word Error Rate: {metrics['wer']}")